In [1]:
import csv
import json
import re
import ast

import pandas as pd
import requests

In [2]:
def getItem(category):
    url = 'https://api.nookplaza.net/items?category={}'.format(category)
    r = requests.get(url)
    text = r.json()
    return text['results']

In [3]:
nookplazaUrl = 'https://nookplaza.net'
r = requests.get(nookplazaUrl)

m = re.search('main\.dart\.js\?v=(\d*)', r.text)
dartJs = m.group()

mainJsUrl = '{}/{}'.format(nookplazaUrl, dartJs)
r = requests.get(mainJsUrl)
mainJs = r.text

# 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)

m = [item.encode("utf-8").decode('unicode-escape') for item in m]
m = [ast.literal_eval(item) for item in m if not 'null' in item]

nameMap = {}
for item in m:
    nameMap[item["en-us"]] = item

print('get {} i18n data'.format(len(nameMap.keys())))

df = pd.read_excel('../tempData/Furniture.xlsx',sheetname=1)

EnTwTable = ['類別', '取得方式', '標籤', '主題', '款式', '樣式', '款式說明', '樣式說明', 'DIY原料']

nameMapCustom = {}
for EnTw in EnTwTable:
    itemType_en = df[EnTw].tolist()
    itemType_tw = df['{}_tw'.format(EnTw)].tolist()
    itemType_en = [x for x in itemType_en if str(x) != 'nan']
    itemType_tw = [x for x in itemType_tw if str(x) != 'nan']
    
    for item in zip(itemType_en, itemType_tw):
        if item[0] in nameMapCustom and item[1] != nameMapCustom[item[0]]['zh-tw']:
            print(item)
            print(nameMapCustom[item[0]])
            print('------\n')
    
        nameMapCustom[item[0]] = {
            'zh-tw': item[1]
        }
        
print('Custom {} i18n data'.format(len(nameMapCustom.keys())))

def en2twFn(en):
    if en.lower() in nameMap:
        return nameMap[en.lower()]

    elif (en[0].lower() + en[1:]) in nameMap:
        return nameMap[(en[0].lower() + en[1:])]
    
    elif en in nameMap:
        return nameMap[en]
    
    elif en in nameMapCustom:
        return nameMapCustom[en]
    
    elif en.lower() in nameMapCustom:
        return nameMapCustom[en.lower()]

    elif (en[0].lower() + en[1:]) in nameMapCustom:
        return nameMapCustom[(en[0].lower() + en[1:])]
    
    else:
        print(en)
        raise en
#         return {'zh-tw': en}

get 5713 i18n data
Custom 832 i18n data


In [4]:
output = []

furnitures = getItem('Furniture')
recipes = getItem('Recipes')

DIYMap = {}
for diy in recipes:
    DIYMap[diy['name']] = diy['content']

for item in furnitures:
    if item["category"] == "Art":
        continue
        
    pickData = {}
    name = en2twFn(item['name'])
    
    pickData['id'] = int(item['content']['internalID'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    pickData['buy'] = item['content']['buy']
    pickData['sell'] = item['content']['sell']

    pickData['category'] = en2twFn(item['content']['category'])['zh-tw']
    pickData['tag'] = en2twFn(item['content']['tag'])['zh-tw']
    pickData['themes'] = [en2twFn(theme)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
    
    pickData['interact'] = item['content']['interact']
    pickData['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['DIY'] = item['content']['dIY']
    pickData['diyInfo'] = {}

    if item['content']['dIY']:
        diyInfo = DIYMap[item['name']]
        pickData['diyInfo']['materials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in diyInfo['materials']]
        pickData['diyInfo']['obtainedFrom'] = [(en2twFn(obtainedFrom)['zh-tw'] if 'villagers' in obtainedFrom else obtainedFrom) for obtainedFrom in diyInfo['obtainedFrom']]
        pickData['diyInfo']['sourceNotes'] = diyInfo['sourceNotes']
        
    pickData['filename'] = item['content']['filename']
    
    pickData['bodyTitle'] = '無' if item['content']['bodyTitle'] is None else en2twFn(item['content']['bodyTitle'])['zh-tw']
    pickData['bodyCustomize'] = item['content']['bodyCustomize']
    
    pickData['patternTitle'] = '無' if item['content']['patternTitle'] is None else en2twFn(item['content']['patternTitle'])['zh-tw']
    pickData['patternCustomize'] = item['content']['patternCustomize']

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        bodyIndex, pattrenIndex = content['filename'].split('_')[-2:]
        
        bodyColor = '' if content['bodyColor'] is None else en2twFn(content['bodyColor'])['zh-tw']
        patternColor = '' if content['patternColor'] is None else en2twFn(content['patternColor'])['zh-tw']
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex
        
    pickData['variations'] = variations
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

In [5]:
with open('../../nook_link/src/data/furniture.json', 'wt') as fout:
    fout.write(json.dumps(output))

with open('furniture.json', 'wt') as fout:
    fout.write(json.dumps(output))

dictName = ['id', 'name_e', 'name_j', 'name_c', 'buy', 'sell', 'category', 'tag', 'themes', 'interact', 'size', 'obtainedFrom', 'DIY', 'diyInfo']
dictName += ['filename', 'bodyTitle', 'patternTitle', 'bodyCustomize', 'patternCustomize', 'variations']

with open('items.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    
    output_csv = [dictName]
    for row in output:
        row_ = [row[name] for name in dictName]
        output_csv.append(row_)
    csvout.writerows(output_csv)